In [ ]:
import mindspore
import mindspore.nn as nn
import torch
from skimage.metrics import peak_signal_noise_ratio as compare_psnr
from skimage.metrics import structural_similarity as compare_ssim

from PIL import Image
import math
import numpy as np
import cv2
    
#定义ESPCN模型
class ESPCN(nn.Cell):
    def __init__(self, scale_factor, num_channels=1):
        super(ESPCN, self).__init__()
        self.first_part = nn.SequentialCell(
            nn.Conv2d(num_channels, 64, kernel_size=5, padding=5//2,pad_mode='pad',has_bias=True),
            nn.Tanh(),
            nn.Conv2d(64, 32, kernel_size=3, padding=3//2,pad_mode='pad',has_bias=True),
            nn.Tanh(),
        )
        self.last_part = nn.SequentialCell(
            nn.Conv2d(32, num_channels * (scale_factor ** 2), kernel_size=3, padding=3 // 2,pad_mode='pad',has_bias=True),
            nn.PixelShuffle(scale_factor)
        )
    def construct(self, x):
        x = self.first_part(x)
        x = self.last_part(x)
        return x
 
#将ycbcr格式图像转为rgb格式
def do_Convert_ycbcr_to_rgb(img, dim_order='hwc'):
    if dim_order == 'hwc':
        r = 298.082 * img[..., 0] / 256. + 408.583 * img[..., 2] / 256. - 222.921
        g = 298.082 * img[..., 0] / 256. - 100.291 * img[..., 1] / 256. - 208.120 * img[..., 2] / 256. + 135.576
        b = 298.082 * img[..., 0] / 256. + 516.412 * img[..., 1] / 256. - 276.836
    else:
        r = 298.082 * img[0] / 256. + 408.583 * img[2] / 256. - 222.921
        g = 298.082 * img[0] / 256. - 100.291 * img[1] / 256. - 208.120 * img[2] / 256. + 135.576
        b = 298.082 * img[0] / 256. + 516.412 * img[1] / 256. - 276.836
    return np.array([r, g, b]).transpose([1, 2, 0])


#将rgb格式图像转为ycbcr格式
def do_Convert_rgb_to_ycbcr(img, dim_order='hwc'):
    if dim_order == 'hwc':
        y = 16. + (64.738 * img[..., 0] + 129.057 * img[..., 1] + 25.064 * img[..., 2]) / 256.
        cb = 128. + (-37.945 * img[..., 0] - 74.494 * img[..., 1] + 112.439 * img[..., 2]) / 256.
        cr = 128. + (112.439 * img[..., 0] - 94.154 * img[..., 1] - 18.285 * img[..., 2]) / 256.
    else:
        y = 16. + (64.738 * img[0] + 129.057 * img[1] + 25.064 * img[2]) / 256.
        cb = 128. + (-37.945 * img[0] - 74.494 * img[1] + 112.439 * img[2]) / 256.
        cr = 128. + (112.439 * img[0] - 94.154 * img[1] - 18.285 * img[2]) / 256.
    return np.array([y, cb, cr]).transpose([1, 2, 0])


#进行图像预处理
def do_Preprocess(img):
    img = np.array(img).astype(np.float32)
    ycbcr = do_Convert_rgb_to_ycbcr(img)
    x = ycbcr[..., 0]
    x /= 255.
    x = mindspore.Tensor(x)
    x = x.unsqueeze(0).unsqueeze(0)
    return x, ycbcr

#利用双三次插值处理测试图像，包括下采样的图像lr_img
def do_Bicubic(test_img,scale):  
    #获取调整后的图像宽高，确保可以被scale整除
    width = (test_img.width // scale) * scale
    height = (test_img.height // scale) * scale
    
    #调用Bicubic
    src_img = test_img.resize((width,height),Image.BICUBIC)   #调整后的原始图像，保证在计算指标时，宽高能够对齐
    lr_img = src_img.resize((src_img.width // scale, src_img.height // scale),Image.BICUBIC)   #获取低分辨率图像，下采样结果
   
    bi_img = lr_img.resize((lr_img.width * scale, lr_img.height * scale),Image.BICUBIC)   #重新resize到原大小，方便计算指标
    
    return src_img, lr_img, bi_img


#调用ESPCN超分辨率算法进行图像处理
def do_ESPCN(path, lr_img, ycbcr): 
    mindspore.set_context(mode=mindspore.GRAPH_MODE)

    # 定义超分辨率模型
    upscale_factor = 3  #放大倍数与模型保持一致
    model = ESPCN(upscale_factor)
    # 加载预训练模型权
    model_state_disk=torch.load(path + 'espcn_x3.pth', map_location=lambda storage, loc: storage)
    for key, parameter in model.parameters_and_names():                # 获取ms模型的参数名和数值
        temp=mindspore.Tensor(model_state_disk.get(key).detach().numpy())
        parameter.set_data(temp)
    
    
        
    # 将模型设置为评估模式  
    model.set_train(False)

    # 对图像进行超分辨率处理
    preds = model(lr_img).clamp(0.0,1.0)

    # 保存处理后的图像
    preds = preds.mul(255.0).numpy().squeeze(0).squeeze(0)

    output = np.array([preds, ycbcr[..., 1], ycbcr[..., 2]]).transpose([1, 2, 0])
    output = np.clip(do_Convert_ycbcr_to_rgb(output), 0.0, 255.0).astype(np.uint8)
    output = Image.fromarray(output)
 
    return output


#计算指标psnr以及ssim
def do_Calculate_Metrics(src_image_path, res_image_path):
    # 加载原始图像和超分辨图像
    src_img = cv2.imread(src_image_path)
    res_img = cv2.imread(res_image_path)
    
    #将图像转为灰度图
    src_img_gray = cv2.cvtColor(src_img, cv2.COLOR_BGR2GRAY)
    res_img_gray = cv2.cvtColor(res_img, cv2.COLOR_BGR2GRAY)
    
    # 计算PSNR和SSIM
    psnr = compare_psnr(src_img_gray, res_img_gray, data_range=255)
    ssim = compare_ssim(src_img_gray, res_img_gray, multichannel=True, data_range=255)
    
    return psnr, ssim

if __name__ =="__main__": 
      
    path = ""
    test_img = Image.open(path + 'get_set5_psnr/Set5/' + 'woman.png').convert('RGB')
    
    
    #使用Bicubic插值进行下采样
    
    scale = 3  #下采样倍数
    src_img, lr_img, bi_img = do_Bicubic(test_img, scale)  #lr_img为下采样后的低分辨率图像， bi_img则为了后续方便比较
    src_img.save(path + "woman_Src.png")
    bi_img.save(path + "woman_Bi.png")
    
    
    #对下采样后的图像进行预处理
    
    lr_img, _ = do_Preprocess(lr_img)
    _, ycbcr = do_Preprocess(bi_img)
    
    
    #使用ESPCN图像超分辨方法处理
    
    output_img = do_ESPCN(path, lr_img, ycbcr)
    output_img.save(path + 'woman_Res.png')  
    
    
    #计算原始图像和超分辨图像之间的PSNR和SSIM
    
    psnr, ssim = do_Calculate_Metrics(path + 'woman_Src.png', path + 'woman_Res.png')
    print("PSNR:", psnr)
    print("SSIM:", ssim)